<img src="https://10botics.com/logo_jnb.png" width="300"/>

In [ ]:
# LED Racing Game Project

## Learning Objectives

In this final project, you will combine everything you've learned to create an exciting LED racing game!

**You will learn:**
1. Integrate button controls with LED strip animations
2. Manage multiple players and game states
3. Create smooth animations and visual effects
4. Implement game logic with timers and scoring
5. Build a complete interactive game system

**Skills from previous lessons:**
- ✅ Button reading and debouncing
- ✅ Basic LED control
- ✅ WS2812 LED strip programming
- ✅ Color theory and animations


In [ ]:
## Game Overview

### What we're building:
- **3-Player LED Racing Game** on a WS2812 LED strip
- Each player controls their racer with a button
- Players race around a circular track (LED strip)
- First player to complete 3 laps wins!
- Winner gets a victory light show

### Game Features:
- 🎮 **3 Player Controls**: Each player has their own button
- 🌈 **Player Colors**: Red, Green, Blue racers
- 🏁 **Lap Counting**: Track grows longer with each lap
- ⏱️ **Game Timer**: 90-second time limit
- 🎊 **Victory Celebration**: Winner's color flashes across the strip
- 🔄 **Auto Restart**: New game starts automatically


In [ ]:
## Hardware Setup

### Required Components:
- Raspberry Pi 4
- WS2812 LED Strip (30+ LEDs recommended)
- 3 Push Buttons
- 3 Pull-down Resistors (10kΩ)
- Breadboard and jumper wires
- External 5V power supply (for longer strips)

### Wiring Diagram:
```
Raspberry Pi    WS2812 Strip
5V      -----> VCC (Power)
GPIO 19 -----> DIN (Data In)
GND     -----> GND (Ground)

Raspberry Pi    Buttons
GPIO 21 -----> Player 1 Button (Red)
GPIO 20 -----> Player 2 Button (Green)  
GPIO 16 -----> Player 3 Button (Blue)
GND     -----> Button ground pins
```

### Wiring Steps:
1. Connect WS2812 strip: VCC→5V, DIN→GPIO19, GND→GND
2. Connect Player 1 button: GPIO21 and GND
3. Connect Player 2 button: GPIO20 and GND
4. Connect Player 3 button: GPIO16 and GND
5. Use pull-down resistors for stable button readings


In [ ]:
## Step 1: Install and Import Libraries

**Objective:** Set up all required libraries for the racing game

**Instructions:** Install and import the libraries we need


In [ ]:
# Install required libraries
!pip install rpi_ws281x RPi.GPIO

print("✅ Libraries installed successfully!")


In [ ]:
# Import all required libraries
import time
import random
import threading
from rpi_ws281x import *
import RPi.GPIO as GPIO

print("✅ All libraries imported successfully!")


In [ ]:
## Step 2: Configure Hardware Settings

**Objective:** Set up GPIO pins and LED strip configuration

**Instructions:** Configure all the hardware parameters


In [ ]:
# LED Strip Configuration
LED_COUNT = 30        # Number of LED pixels (adjust for your strip)
LED_PIN = 19         # GPIO pin connected to the pixels
LED_FREQ_HZ = 800000 # LED signal frequency in hertz
LED_DMA = 10         # DMA channel to use for generating signal
LED_INVERT = False   # True to invert the signal
LED_CHANNEL = 1      # Set to '1' for GPIO 19

# Button GPIO Pins
BUTTON_PLAYER_1 = 21  # Red player
BUTTON_PLAYER_2 = 20  # Green player
BUTTON_PLAYER_3 = 16  # Blue player

# Game Settings
NUM_PLAYERS = 3
LAPS_TO_WIN = 3
GAME_DURATION = 90    # seconds
TRAIL_LENGTH = 3      # LEDs per player trail

print("✅ Hardware configuration ready!")
print(f"LED Count: {LED_COUNT}")
print(f"LED Pin: GPIO {LED_PIN}")
print(f"Players: {NUM_PLAYERS}")
print(f"Laps to win: {LAPS_TO_WIN}")


In [ ]:
## Step 3: Initialize Hardware

**Objective:** Set up GPIO and LED strip objects

**Instructions:** Initialize the hardware components


In [ ]:
# Setup GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(BUTTON_PLAYER_1, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
GPIO.setup(BUTTON_PLAYER_2, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
GPIO.setup(BUTTON_PLAYER_3, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)

# Create LED strip object
strip = Adafruit_NeoPixel(LED_COUNT, LED_PIN, LED_FREQ_HZ, LED_DMA, LED_INVERT, LED_CHANNEL)
strip.begin()

print("✅ Hardware initialized!")
print("✅ GPIO pins configured")
print("✅ LED strip ready")


In [ ]:
## Step 4: Create Game Logic

**Objective:** Build the core game mechanics and player management

**Instructions:** Create the RacingGame class to manage players and game state


In [ ]:
# Player colors (RGB values)
PLAYER_COLORS = [
    Color(255, 0, 0),    # Player 1: Red
    Color(0, 255, 0),    # Player 2: Green  
    Color(0, 0, 255),    # Player 3: Blue
]

# Game state class
class RacingGame:
    def __init__(self):
        self.player_positions = [0, 0, 0]  # Current LED position for each player
        self.player_laps = [0, 0, 0]       # Completed laps for each player
        self.game_start_time = 0
        self.game_active = False
        self.winner = -1
        
    def reset_game(self):
        """Reset all game variables for a new game"""
        self.player_positions = [0, 0, 0]
        self.player_laps = [0, 0, 0]
        self.game_start_time = time.time()
        self.game_active = True
        self.winner = -1
        print("🎮 New game started!")
        
    def move_player(self, player_id, steps=1):
        """Move a player forward by specified steps"""
        if not self.game_active or player_id >= NUM_PLAYERS:
            return
            
        old_position = self.player_positions[player_id]
        new_position = old_position + steps
        
        # Check if player completed a lap
        if new_position >= LED_COUNT:
            self.player_laps[player_id] += 1
            new_position = new_position % LED_COUNT
            print(f"🏁 Player {player_id + 1} completed lap {self.player_laps[player_id]}!")
            
        self.player_positions[player_id] = new_position
        
    def check_winner(self):
        """Check if any player won or time is up"""
        # Check lap-based winner
        for i, laps in enumerate(self.player_laps):
            if laps >= LAPS_TO_WIN:
                self.winner = i
                self.game_active = False
                return True
                
        # Check time-based winner
        if time.time() - self.game_start_time >= GAME_DURATION:
            # Find player with most laps, or furthest position if tied
            max_laps = max(self.player_laps)
            leaders = [i for i, laps in enumerate(self.player_laps) if laps == max_laps]
            
            if len(leaders) == 1:
                self.winner = leaders[0]
            else:
                # Tie-breaker: furthest position
                furthest_pos = max(self.player_positions[i] for i in leaders)
                self.winner = next(i for i in leaders if self.player_positions[i] == furthest_pos)
                
            self.game_active = False
            return True
            
        return False

# Create game instance
game = RacingGame()

print("✅ Game logic ready!")
print("✅ Player management system created")
print("✅ Win conditions implemented")


In [ ]:
## Step 5: Create Visual Effects

**Objective:** Build stunning visual effects and animations

**Instructions:** Create functions to display players, animations, and celebrations


In [ ]:
def clear_strip():
    """Turn off all LEDs"""
    for i in range(LED_COUNT):
        strip.setPixelColor(i, Color(0, 0, 0))
    strip.show()

def display_players():
    """Display all players on the LED strip with trails"""
    clear_strip()
    
    for player_id in range(NUM_PLAYERS):
        position = game.player_positions[player_id]
        laps = game.player_laps[player_id]
        color = PLAYER_COLORS[player_id]
        
        # Calculate trail length (grows with laps)
        trail_length = min(TRAIL_LENGTH + laps, 8)
        
        # Draw player trail
        for i in range(trail_length):
            trail_pos = (position - i) % LED_COUNT
            brightness = 255 - (i * 30)  # Fade trail
            if brightness < 50:
                brightness = 50
                
            # Scale color by brightness
            r = (color >> 16) & 0xFF
            g = (color >> 8) & 0xFF
            b = color & 0xFF
            
            scaled_color = Color(
                (r * brightness) // 255,
                (g * brightness) // 255, 
                (b * brightness) // 255
            )
            
            strip.setPixelColor(trail_pos, scaled_color)
    
    strip.show()

def countdown_animation():
    """3-2-1 countdown animation"""
    print("🚦 Starting countdown...")
    
    for count in range(3, 0, -1):
        print(f"   {count}...")
        clear_strip()
        
        # Fill strip with countdown color
        if count == 3:
            color = Color(255, 0, 0)  # Red
        elif count == 2:
            color = Color(255, 165, 0)  # Orange
        else:
            color = Color(255, 255, 0)  # Yellow
            
        for i in range(count * (LED_COUNT // 3)):
            strip.setPixelColor(i, color)
        strip.show()
        time.sleep(1)
    
    # GO! - Flash green
    print("   GO! 🏁")
    for i in range(LED_COUNT):
        strip.setPixelColor(i, Color(0, 255, 0))
    strip.show()
    time.sleep(0.5)
    clear_strip()

def victory_celebration(winner_id):
    """Celebrate the winner with flashing lights"""
    winner_color = PLAYER_COLORS[winner_id]
    print(f"🎊 Player {winner_id + 1} wins! Celebrating...")
    
    # Flash winner's color 10 times
    for flash in range(10):
        # Fill with winner color
        for i in range(LED_COUNT):
            strip.setPixelColor(i, winner_color)
        strip.show()
        time.sleep(0.3)
        
        # Turn off
        clear_strip()
        time.sleep(0.2)
    
    # Final rainbow celebration
    print("🌈 Rainbow finale!")
    for j in range(256):
        for i in range(LED_COUNT):
            pixel_hue = (i * 256 // LED_COUNT + j) % 256
            strip.setPixelColor(i, wheel(pixel_hue))
        strip.show()
        time.sleep(0.02)
    
    clear_strip()

def wheel(pos):
    """Generate rainbow colors across 0-255 positions"""
    if pos < 85:
        return Color(pos * 3, 255 - pos * 3, 0)
    elif pos < 170:
        pos -= 85
        return Color(255 - pos * 3, 0, pos * 3)
    else:
        pos -= 170
        return Color(0, pos * 3, 255 - pos * 3)

print("✅ Visual effects ready!")
print("   - Player trails with fade effect")
print("   - Countdown animation")
print("   - Victory celebration")
print("   - Rainbow effects")


In [ ]:
## Step 6: Handle Player Input

**Objective:** Create responsive button controls with proper debouncing

**Instructions:** Build the input system for player controls


In [ ]:
# Button state tracking for debouncing
button_states = {}
last_press_times = {}
DEBOUNCE_TIME = 0.1  # 100ms debounce

def init_button_tracking():
    """Initialize button state tracking"""
    global button_states, last_press_times
    button_pins = [BUTTON_PLAYER_1, BUTTON_PLAYER_2, BUTTON_PLAYER_3]
    
    for pin in button_pins:
        button_states[pin] = False
        last_press_times[pin] = 0

def check_button_press(pin, player_id):
    """Check if button was pressed with debouncing"""
    current_time = time.time()
    current_state = GPIO.input(pin)
    
    # Check for button press (low to high transition with pull-down)
    if (current_state and not button_states[pin] and 
        current_time - last_press_times[pin] > DEBOUNCE_TIME):
        
        button_states[pin] = True
        last_press_times[pin] = current_time
        return True
    
    button_states[pin] = current_state
    return False

def handle_player_inputs():
    """Check all player button inputs"""
    if not game.game_active:
        return
        
    # Check Player 1 button
    if check_button_press(BUTTON_PLAYER_1, 0):
        game.move_player(0, 2)  # Move 2 steps
        print(f"🔴 Player 1 moved! Position: {game.player_positions[0]}")
    
    # Check Player 2 button  
    if check_button_press(BUTTON_PLAYER_2, 1):
        game.move_player(1, 2)  # Move 2 steps
        print(f"🟢 Player 2 moved! Position: {game.player_positions[1]}")
    
    # Check Player 3 button
    if check_button_press(BUTTON_PLAYER_3, 2):
        game.move_player(2, 2)  # Move 2 steps
        print(f"🔵 Player 3 moved! Position: {game.player_positions[2]}")

# Initialize button tracking
init_button_tracking()

print("✅ Input system ready!")
print("   - Button debouncing implemented")
print("   - 3 player inputs configured")
print("   - Move distance: 2 LEDs per button press")


In [ ]:
## Step 7: Main Game Loop

**Objective:** Create the main game execution loop

**Instructions:** Put everything together in the complete game loop


In [ ]:
def run_racing_game():
    """Main game loop"""
    print("🏁 LED Racing Game Starting!")
    print("Press Ctrl+C to stop")
    print()
    
    try:
        while True:
            # Start new game
            print("\n" + "="*50)
            print("🎮 STARTING NEW RACE!")
            print("="*50)
            
            game.reset_game()
            countdown_animation()
            
            # Game loop
            while game.game_active:
                # Handle player inputs
                handle_player_inputs()
                
                # Update display
                display_players()
                
                # Check for winner
                if game.check_winner():
                    break
                
                # Show game status every 10 seconds
                elapsed = time.time() - game.game_start_time
                if int(elapsed) % 10 == 0 and elapsed > 0:
                    remaining = GAME_DURATION - elapsed
                    if remaining > 0:
                        print(f"⏱️  Time remaining: {remaining:.0f}s")
                        for i, (pos, laps) in enumerate(zip(game.player_positions, game.player_laps)):
                            print(f"   Player {i+1}: Lap {laps+1}, Position {pos}")
                
                time.sleep(0.05)  # 20 FPS update rate
            
            # Game ended - show results
            print("\n🏁 RACE FINISHED!")
            if game.winner >= 0:
                player_names = ["Red", "Green", "Blue"]
                print(f"🎊 Winner: Player {game.winner + 1} ({player_names[game.winner]})!")
                print(f"   Completed {game.player_laps[game.winner]} laps")
                
                # Show final standings
                print("\n📊 Final Standings:")
                standings = [(i, game.player_laps[i], game.player_positions[i]) 
                           for i in range(NUM_PLAYERS)]
                standings.sort(key=lambda x: (x[1], x[2]), reverse=True)
                
                for rank, (player_id, laps, pos) in enumerate(standings, 1):
                    print(f"   {rank}. Player {player_id+1} - {laps} laps, position {pos}")
                
                victory_celebration(game.winner)
            
            print("\n⏳ Next race starting in 5 seconds...")
            time.sleep(5)
            
    except KeyboardInterrupt:
        print("\n\n🛑 Game stopped by user")
    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        # Cleanup
        clear_strip()
        GPIO.cleanup()
        print("✅ Hardware cleaned up")
        print("Thanks for playing LED Racing Game! 🏁")

print("✅ Main game loop ready!")
print("   - Automatic game restart")
print("   - Real-time status updates")  
print("   - Final standings display")
print("   - Proper cleanup on exit")


In [ ]:
## Step 8: Test Individual Components

**Objective:** Test each system before running the full game

**Instructions:** Run these tests to verify everything works correctly


In [ ]:
# Test 1: LED Strip Rainbow Test
print("🧪 Test 1: LED Strip Rainbow")
for j in range(256):
    for i in range(LED_COUNT):
        pixel_hue = (i * 256 // LED_COUNT + j) % 256
        strip.setPixelColor(i, wheel(pixel_hue))
    strip.show()
    time.sleep(0.02)

clear_strip()
print("✅ LED strip test complete!")


In [ ]:
# Test 2: Button Input Test
print("🧪 Test 2: Button Input Test")
print("Press each button to test (20 second test)")
print("Player 1: Red LED | Player 2: Green LED | Player 3: Blue LED")

start_time = time.time()
while time.time() - start_time < 20:
    # Test button presses with colored feedback
    if GPIO.input(BUTTON_PLAYER_1):
        strip.setPixelColor(0, Color(255, 0, 0))  # Red
        print("🔴 Player 1 button pressed!")
    elif GPIO.input(BUTTON_PLAYER_2):
        strip.setPixelColor(0, Color(0, 255, 0))  # Green
        print("🟢 Player 2 button pressed!")
    elif GPIO.input(BUTTON_PLAYER_3):
        strip.setPixelColor(0, Color(0, 0, 255))  # Blue
        print("🔵 Player 3 button pressed!")
    else:
        strip.setPixelColor(0, Color(0, 0, 0))  # Off
    
    strip.show()
    time.sleep(0.1)

clear_strip()
print("✅ Button test complete!")


In [ ]:
## Step 9: Run the Complete Game! 🏁

**Objective:** Launch the full LED racing game

### Game Instructions for Players:
1. **3 players maximum** (Red, Green, Blue)
2. **Press your button** to move your racer forward 2 LEDs
3. **Complete 3 laps** to win the race
4. **Watch your trail** grow longer with each completed lap
5. **90-second time limit** - furthest player wins if no one finishes
6. **Enjoy the victory show** when someone wins!

### Tips for Success:
- Rapid button pressing moves you faster
- Watch other players' progress on the strip
- Position 0 is the start/finish line
- Press **Ctrl+C** to stop the game safely

**Ready to race? Execute the cell below!**


In [ ]:
# 🏁 START THE LED RACING GAME! 🏁
run_racing_game()


In [ ]:
## Step 10: Cleanup and Customization

**Objective:** Clean up resources and explore enhancements

**Always run cleanup when finished to properly release GPIO resources**


In [ ]:
# Cleanup function (run this if game stops unexpectedly)
def cleanup():
    """Clean up GPIO and turn off LEDs"""
    clear_strip()
    GPIO.cleanup()
    print("✅ Cleanup complete!")

# Run cleanup
cleanup()


In [ ]:
## Project Summary & Customization Ideas

**Congratulations!** You've successfully built a complete LED Racing Game! 🎉

### What You've Accomplished:
1. ✅ **Hardware Integration**: Combined buttons, GPIO, and LED strips
2. ✅ **Game Logic**: Implemented player movement, lap counting, and win conditions
3. ✅ **Visual Effects**: Created smooth animations and victory celebrations
4. ✅ **User Interface**: Built responsive button controls with debouncing
5. ✅ **System Design**: Structured code with classes and modular functions
6. ✅ **Error Handling**: Added proper cleanup and exception handling

### Skills Mastered:
- **GPIO Programming**: Button inputs with pull-down resistors
- **LED Control**: WS2812 strip programming with colors and effects
- **Game Development**: State management, timing, and player interaction
- **Real-time Systems**: Smooth animations and responsive controls
- **Python Programming**: Classes, functions, and hardware libraries

### Easy Customization Ideas:
1. **Change Colors**: Modify `PLAYER_COLORS` for different player colors
2. **Adjust Speed**: Change move distance in `handle_player_inputs()`
3. **Game Duration**: Modify `GAME_DURATION` for longer/shorter games
4. **Trail Length**: Adjust `TRAIL_LENGTH` for longer player trails
5. **Win Condition**: Change `LAPS_TO_WIN` for more/fewer laps

### Advanced Customization Ideas:
1. **Power-ups**: Add special LEDs that give speed boosts
2. **Obstacles**: Create "slow zones" that reduce movement
3. **Sound Effects**: Add buzzer sounds for button presses and wins
4. **More Players**: Expand to 4+ players with additional buttons
5. **Different Tracks**: Create spiral or back-and-forth track layouts
6. **Difficulty Modes**: Add AI players with different speeds
7. **Tournament Mode**: Multi-round elimination tournaments

### Technical Concepts Learned:
- **Hardware Interfacing**: Raspberry Pi GPIO and WS2812 protocol
- **Event-driven Programming**: Button press handling and game events
- **Color Theory**: RGB values and color wheel mathematics
- **Animation**: Frame-based updates and visual effects
- **Game Design**: Rules, scoring, and player feedback

### Next Steps:
- Try the customization ideas above
- Build other LED-based projects
- Explore advanced Raspberry Pi programming
- Share your game with friends and family!
- Consider making a tournament bracket system
- Add a web interface for remote control

**Excellent work building your LED Racing Game!** 🏁🌈✨

*You've successfully combined all the skills from button basics, LED control, and WS2812 programming into one amazing interactive project!*


<hr/>

## Congratulation! You have finished this chapter.

This jupyter notebook is created by 10Botics. <br>
For permission to use in school, please contact info@10botics.com <br>
All rights reserved. 2025.